# Syntax Analysis 

This notebook demonstrates the syntax analysis features of `nbtest`. The purpose of syntax analysis is to provide an easy way to introspect the tested code. The introspection uses Pythons excellent `ast` package so there's no dependence on fragile regular expressions. 

In [1]:
%load_ext nbtest

In [2]:
"""
Firstname Lastname
@answer1
"""

a = 100 
b = 200 

print("Hello World")
type(a < b)

def myfunc(a, b, *x, y, **z):
    """This is myfunc."""
    print("myfunc")
    c = 300
    d = 400 

    def inner_myfunc(inner_a, inner_b):
        """An inner docstring"""
        print("inner_myfunc")
        name = "FOO".lower()
        return name + inner_a + inner_b
            
    return a + b + c + d

class MyClass:
    """This is MyClass"""
    E = 500
    F = 600 

    class InnerClass:
        pass 

    def member(self, g, h):
        """Member function"""
        print("member")
        i = 700
        j = 800 
        return self.E + self.F + g + h + i + j

Hello World


## Analysis API

The `TagCacheEntry` class inherits the AnalysisAPI. `AnalysisNode`s can also be constructed on arbitrary source code.

In [3]:
import nbtest
from nbtest.analysis import AnalysisNode

entry = nbtest.get("@answer1")
assert isinstance(entry, AnalysisNode)

n = AnalysisNode("""print("Hello World")""")
help(n)

Help on AnalysisNode in module nbtest.analysis object:

class AnalysisNode(builtins.object)
 |  AnalysisNode(source, tree=None)
 |
 |  An API for simple syntax analysis.
 |
 |  Methods defined here:
 |
 |  __init__(self, source, tree=None)
 |      Create an analysis node with optional source code.
 |
 |      source: Source code.
 |      tree: The parse tree or subtree corresponding to the source. If tree is
 |          None it will be generated using ast.parse(source)
 |
 |  count_assignments(self, name) -> set[str]
 |      Count the number of times the variable `name` is assigned.
 |
 |  count_calls(self, name) -> set[str]
 |      Count the number of times the function `name` is called.
 |
 |  count_references(self, name) -> set[str]
 |      Count the number of times the symbol `name` is referenced.
 |
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |
 |  arguments
 |      If the current node is a function definition, t

## Access to the Source and Docstrings

The API provides access to the source and docstring for convenience.

In [4]:
import nbtest 
import re 

answer1 = nbtest.get("@answer1")

assert re.search(r"InnerClass", answer1.source) is not None, """I can't find InnerClass (the silly way)"""
assert re.search(r"Firstname", answer1.docstring) is not None, """I should see your name in the docstring."""

## Syntax Queries 

The api in `analysis.py` seeks to make most questions about the syntax of a solution easy to write. It's simple to write a test that looks for the presence or the absence of a parse token:

In [5]:
import ast

answer1 = nbtest.get("@answer1")

assert ast.Lt in answer1.tokens, """The solution must have the < operator."""
assert ast.Del not in answer1.tokens, """You cannot use the del operator."""

The `assignments` attribute contains the `set` of names of variables assigned at the root level of the parse tree. 

In [6]:
assert 'a' in answer1.assignments, """You should assign the vairable a."""
assert 'b' in answer1.assignments, """You should assign the vairable b."""
assert 'c' not in answer1.assignments, """You should NOT assign the vairable c."""

The `constants` attribute contains `the` set of all literal constants at the root level of the parse tree.  

In [7]:
assert 100 in answer1.constants, """You should have the constant 100 in your code."""
assert 300 not in answer1.constants, """You should NOT have the constant 300 in your code."""
assert 500 not in answer1.constants, """You should NOT have the constant 500 in your code."""

The `calls` attribute contains the `set` of all function calls at the root level of the parse tree.

In [8]:
assert 'print' in answer1.calls, """You should use print()"""
assert 'lower' not in answer1.calls, """You cannot use lower()"""

The `functions` attribute returns a `dict` of all function **definitions** at the root level of the parse tree. The dictionary values are `AnalysisNode`s that can be further inspected.

In [9]:
myfunc = answer1.functions["myfunc"]

# All of the previous attributes work. 
assert myfunc.docstring is not None, """The function has no docstring."""
assert "c" in myfunc.assignments, """The function should assign c"""
assert "name" not in myfunc.assignments, """The function should NOT assign name."""

# Function arguments. 
assert "a" in myfunc.arguments, """The function should have an argument named a"""
assert "**z" in myfunc.arguments, """The function should have the generic kw argument named z"""

# Inner functions are accessible. 
inner = myfunc.functions["inner_myfunc"]
assert "name" in inner.assignments, """The inner function should assign name."""

The `classes` attribute finds all of the class definitions at the root level of the parse tree:

In [10]:
my_class = answer1.classes["MyClass"]
assert my_class.docstring is not None, """MyClass should have a docstring."""
assert "E" in my_class.assignments,  """MyClass must define the class variable E"""

assert "member" in my_class.functions, """MyClass should have a function called member()"""
member = my_class.functions["member"]
assert member.docstring is not None, """Member functions should have a docstring"""

## Countables 

Some of the syntax attributes have count methods that allow you to count the number
of items with a particular name.

In [18]:
assert 1 == answer1.count_assignments("a"), """The variable a should be assigned once."""
assert 1 == answer1.count_references("b"), """The variable b should only be used once."""
assert 1 == answer1.count_calls("print"), """You should only call print() once."""

## Tree Access 

You can write your own analysis methods using direct access to the parse tree.

In [11]:
import ast 

# Write your own visitors with tree 
class FindNames(ast.NodeVisitor):
    def visit_Name(self, node: ast.Name):
        print("Found:", node.id)

FindNames().visit(answer1.tree)

Found: a
Found: b
Found: print
Found: type
Found: a
Found: b
Found: print
Found: c
Found: d
Found: print
Found: name
Found: name
Found: inner_a
Found: inner_b
Found: a
Found: b
Found: c
Found: d
Found: E
Found: F
Found: print
Found: i
Found: j
Found: self
Found: self
Found: g
Found: h
Found: i
Found: j
